# CI/CD Pipeline for MedPix Medical Data Analysis Project

This notebook demonstrates how to build a comprehensive CI/CD pipeline for pull requests, including test files and automated workflows for our medical data visualization project.

## 1. Set Up Project Structure

First, let's understand our current project structure and create the necessary directories for CI/CD.

In [ ]:
import os
import json
from pathlib import Path

# Define project structure
project_structure = {
    'medpix-explorer/': 'Vue.js frontend application',
    'Ml-Notebook/': 'Machine learning notebooks and data processing',
    'data/': 'Medical dataset storage',
    'docs/': 'Project documentation',
    '.github/workflows/': 'GitHub Actions CI/CD workflows',
    'tests/': 'Additional test files'
}

print("📁 Project Structure:")
for folder, description in project_structure.items():
    print(f"├── {folder:<20} - {description}")

## 2. Create Test Files with pytest

Let's create comprehensive test files for different components of our project.

In [ ]:
# Example test for data validation
test_data_validation = '''
import pytest
import json
import os
from pathlib import Path

class TestDataValidation:
    """Test medical dataset integrity and structure"""
    
    def setup_method(self):
        """Setup test environment"""
        self.data_path = Path("../data/archive/Cases.json")
        
    def test_cases_json_exists(self):
        """Test that Cases.json file exists"""
        assert self.data_path.exists(), "Cases.json file not found"
        
    def test_cases_json_structure(self):
        """Test the structure of Cases.json"""
        if self.data_path.exists():
            with open(self.data_path, 'r') as f:
                data = json.load(f)
            
            assert isinstance(data, list), "Cases.json should contain a list"
            assert len(data) > 0, "Cases.json should not be empty"
            
            # Test first case structure
            first_case = data[0]
            required_fields = ["URL", "Case Folder", "Image Paths"]
            
            for field in required_fields:
                assert field in first_case, f"Missing required field: {field}"
                
    def test_image_paths_validity(self):
        """Test that image paths are properly formatted"""
        if self.data_path.exists():
            with open(self.data_path, 'r') as f:
                data = json.load(f)
            
            # Test a sample of cases
            sample_cases = data[:10]
            
            for case in sample_cases:
                image_paths = case.get("Image Paths", [])
                assert isinstance(image_paths, list), "Image Paths should be a list"
                
                for path in image_paths:
                    assert isinstance(path, str), "Each image path should be a string"
                    assert path.endswith(('.jpg', '.jpeg', '.png')), f"Invalid image format: {path}"
'''

print("🧪 Sample Test File for Data Validation:")
print(test_data_validation[:500] + "...")

In [ ]:
# Example test for ML notebook functionality
test_ml_notebooks = '''
import pytest
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import os

class TestMLNotebooks:
    """Test ML notebooks execution and output validation"""
    
    def setup_method(self):
        """Setup test environment"""
        self.notebook_dir = Path("../Ml-Notebook")
        
    def test_notebooks_exist(self):
        """Test that notebook directory exists"""
        assert self.notebook_dir.exists(), "ML-Notebook directory not found"
        
    def test_requirements_file(self):
        """Test that requirements.txt exists and is valid"""
        req_file = self.notebook_dir / "requirements.txt"
        
        if req_file.exists():
            with open(req_file, 'r') as f:
                requirements = f.read().strip().split('\n')
            
            # Check for essential packages
            essential_packages = ['numpy', 'pandas', 'matplotlib', 'jupyter']
            req_text = '\n'.join(requirements)
            
            for package in essential_packages:
                assert package in req_text, f"Missing essential package: {package}"
                
    def test_notebook_execution(self):
        """Test that notebooks can be executed without errors"""
        notebook_files = list(self.notebook_dir.glob("*.ipynb"))
        
        for notebook_path in notebook_files:
            with open(notebook_path, 'r') as f:
                nb = nbformat.read(f, as_version=4)
            
            # Skip execution test if notebook is too large
            if len(nb.cells) > 50:
                pytest.skip(f"Skipping large notebook: {notebook_path.name}")
            
            ep = ExecutePreprocessor(timeout=300, kernel_name='python3')
            
            try:
                ep.preprocess(nb, {'metadata': {'path': str(notebook_path.parent)}})
            except Exception as e:
                pytest.fail(f"Notebook {notebook_path.name} failed to execute: {str(e)}")
'''

print("🧪 Sample Test File for ML Notebooks:")
print(test_ml_notebooks[:500] + "...")

## 3. Configure GitHub Actions Workflow

Here's our comprehensive GitHub Actions workflow that runs on pull requests.

In [ ]:
github_workflow = '''
name: CI/CD Pipeline

on:
  pull_request:
    branches: [master, main]
  push:
    branches: [master, main]

jobs:
  test-frontend:
    name: Test Vue.js Frontend
    runs-on: ubuntu-latest
    
    defaults:
      run:
        working-directory: ./medpix-explorer
    
    strategy:
      matrix:
        node-version: [20.x, 22.x]
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v4
      
      - name: Setup Node.js ${{ matrix.node-version }}
        uses: actions/setup-node@v4
        with:
          node-version: ${{ matrix.node-version }}
          cache: 'npm'
          cache-dependency-path: './medpix-explorer/package-lock.json'
      
      - name: Install dependencies
        run: npm ci
      
      - name: Run linting
        run: npm run lint
      
      - name: Check TypeScript types
        run: npm run type-check
      
      - name: Run unit tests
        run: npm run test:unit
      
      - name: Build application
        run: npm run build
'''

print("⚙️ GitHub Actions Workflow:")
print(github_workflow)

## 4. Implement Code Quality Checks

Let's add comprehensive code quality checks including linting, formatting, and security scanning.

In [ ]:
quality_checks = '''
  quality-checks:
    name: Code Quality & Security
    runs-on: ubuntu-latest
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v4
      
      - name: Setup Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.11'
      
      - name: Install Python quality tools
        run: |
          pip install flake8 black isort safety bandit mypy
      
      - name: Run Black formatter check
        run: black --check --diff .
      
      - name: Run isort import sorting check
        run: isort --check-only --diff .
      
      - name: Run flake8 linting
        run: flake8 . --count --select=E9,F63,F7,F82 --show-source --statistics
      
      - name: Run security checks with bandit
        run: bandit -r . -f json -o bandit-report.json || true
      
      - name: Run safety check for vulnerabilities
        run: safety check --json --output safety-report.json || true
      
      - name: Run Trivy vulnerability scanner
        uses: aquasecurity/trivy-action@master
        with:
          scan-type: 'fs'
          scan-ref: '.'
          format: 'sarif'
          output: 'trivy-results.sarif'
'''

print("🔍 Code Quality Checks:")
print(quality_checks)

In [ ]:
# Create configuration files for quality tools
configs = {
    'pyproject.toml': '''
[tool.black]
line-length = 88
target-version = ['py311']
include = '\.pyi?$'
extend-exclude = '''
    # A regex preceded by a single or double slash
    (
      /(
          \.eggs
        | \.git
        | \.hg
        | \.mypy_cache
        | \.tox
        | \.venv
        | _build
        | buck-out
        | build
        | dist
      )/
    )
    '''

[tool.isort]
profile = "black"
line_length = 88

[tool.mypy]
python_version = "3.11"
warn_return_any = true
warn_unused_configs = true
disallow_untyped_defs = true
''',
    
    '.flake8': '''
[flake8]
max-line-length = 88
extend-ignore = E203, E266, E501, W503
max-complexity = 10
select = B,C,E,F,W,T4,B9
'''
}

print("📋 Configuration Files for Quality Tools:")
for filename, content in configs.items():
    print(f"\n📄 {filename}:")
    print(content[:200] + "...")

## 5. Add Branch Protection Rules

Configure repository settings to enforce CI checks before merging.

In [ ]:
branch_protection_guide = '''
## Branch Protection Rules Setup

### 1. Navigate to Repository Settings
- Go to your GitHub repository
- Click on "Settings" tab
- Select "Branches" from the left sidebar

### 2. Add Branch Protection Rule
- Click "Add rule"
- Branch name pattern: `master` (or `main`)

### 3. Configure Protection Settings
✅ Require a pull request before merging
  ✅ Require approvals: 1
  ✅ Dismiss stale PR approvals when new commits are pushed
  ✅ Require review from code owners

✅ Require status checks to pass before merging
  ✅ Require branches to be up to date before merging
  Required status checks:
    - Test Vue.js Frontend (20.x)
    - Test Vue.js Frontend (22.x) 
    - Test ML Notebooks
    - Code Quality & Security
    - Quality Gate

✅ Require conversation resolution before merging
✅ Require signed commits
✅ Require linear history
✅ Include administrators

### 4. Additional Settings
- Allow force pushes: ❌ Disabled
- Allow deletions: ❌ Disabled
'''

print("🛡️ Branch Protection Rules Guide:")
print(branch_protection_guide)

## 6. Test the CI/CD Pipeline

Let's create a comprehensive test strategy to validate our CI/CD pipeline.

In [ ]:
testing_strategy = {
    "Unit Tests": {
        "description": "Test individual components in isolation",
        "tools": ["Vitest (Vue.js)", "pytest (Python)"],
        "coverage": "Components, utilities, stores, data validation"
    },
    
    "Integration Tests": {
        "description": "Test component interactions",
        "tools": ["Vue Test Utils", "pytest fixtures"],
        "coverage": "Router navigation, API calls, data flow"
    },
    
    "End-to-End Tests": {
        "description": "Test complete user workflows",
        "tools": ["Playwright", "Cypress"],
        "coverage": "User journeys, medical case exploration"
    },
    
    "Performance Tests": {
        "description": "Test application performance",
        "tools": ["Lighthouse CI", "Bundle analyzer"],
        "coverage": "Load times, bundle size, memory usage"
    },
    
    "Security Tests": {
        "description": "Test for security vulnerabilities",
        "tools": ["Trivy", "Bandit", "Safety"],
        "coverage": "Dependencies, code patterns, container security"
    }
}

print("🧪 Comprehensive Testing Strategy:")
print("=" * 50)

for test_type, details in testing_strategy.items():
    print(f"\n📋 {test_type}")
    print(f"   Description: {details['description']}")
    print(f"   Tools: {', '.join(details['tools'])}")
    print(f"   Coverage: {details['coverage']}")

In [ ]:
# Sample test commands for different scenarios
test_commands = {
    "Local Development": [
        "npm run test:unit",
        "npm run lint", 
        "npm run type-check",
        "python -m pytest Ml-Notebook/"
    ],
    
    "Pull Request Validation": [
        "npm ci && npm run build",
        "npm run test:unit -- --coverage",
        "python -m pytest --cov=. --cov-report=xml",
        "safety check",
        "bandit -r . -f json"
    ],
    
    "Pre-deployment": [
        "npm run build",
        "npm run preview",
        "python test_notebooks.py",
        "trivy fs ."
    ]
}

print("⚡ Test Commands by Scenario:")
print("=" * 40)

for scenario, commands in test_commands.items():
    print(f"\n🎯 {scenario}:")
    for i, cmd in enumerate(commands, 1):
        print(f"   {i}. {cmd}")

## Summary

This notebook has demonstrated how to build a comprehensive CI/CD pipeline for our MedPix medical data visualization project. The pipeline includes:

### ✅ What We've Implemented:
1. **Automated Testing** - Unit, integration, and security tests
2. **Code Quality Checks** - Linting, formatting, and type checking
3. **Multi-environment Support** - Testing across Node.js versions
4. **Security Scanning** - Vulnerability detection in dependencies
5. **Branch Protection** - Enforced quality gates before merging

### 🚀 Next Steps:
1. Create actual test files based on the examples
2. Set up branch protection rules in GitHub
3. Test the pipeline with a sample pull request
4. Add deployment automation for successful builds
5. Set up monitoring and notifications

The CI/CD pipeline ensures that every pull request maintains code quality, passes all tests, and meets security standards before being merged into the main branch.